# **Data Preprocessing**

In this notebook we aim to prepare the data for modelling.
Data preprocessing will take place in 6 stages:

* **1. Loading in the Data**
* **2. Renaming the Dataframe Fields**
* **3. Merging the Dataframes**
* **4. Creating New Variables**
* **5. Filling in Missing Values**
* **6. Adding Dummy Variables**

After preprocessing is complete, the data will be ready for training a machine learning model 

## **Dependencies**

In [28]:
import pandas as pd
import numpy as np

from dateutil.relativedelta import *

## **1. Loading in the Data**

We are going to read in our six dataframes directly from Github, which will prevent other users from having to locally download the csv files

In [3]:
race_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/race.csv'
races = pd.read_csv(race_url)

results_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/results.csv'
results = pd.read_csv(results_url)

qualifying_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/qualifying_results.csv'
qualifying = pd.read_csv(qualifying_url)

driver_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/driver_standings.csv'
driver_standings = pd.read_csv(driver_url)

constructor_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/constructor_standings.csv'
constructor_standings = pd.read_csv(constructor_url)

weather_url = 'https://raw.githubusercontent.com/DeanLundie/Formula-1/master/Data/weather_info.csv'
weather = pd.read_csv(weather_url)

## **2. Renaming the Dataframe Fields**

Before we can merge the dataframes, we need to rename any fields that have differing names:

In [14]:
qualifying.rename(columns = {'grid_position': 'grid'}, inplace = True)

print(qualifying.shape)
qualifying.head()

(14559, 6)


,grid,driver_name,car,qualifying_time,season,round
0,1,Keke Rosberg ROS,Williams Honda,1:34.526,1983,1
1,2,Alain Prost PRO,Renault,1:34.672,1983,1
2,3,Patrick Tambay TAM,Ferrari,1:34.758,1983,1
3,4,Nelson Piquet PIQ,Brabham BMW,1:35.114,1983,1
4,5,Derek Warwick WAR,Toleman Hart,1:35.206,1983,1


## **3. Merging the Dataframes**

We can now begin merging the dataframes. Since there is no common identifier (which is one-to-one) between the dataframes, we will iteratively add two dataframes together which have a common key. This will be done until all six dataframes are contained within a single dataframe:

In [26]:
df1 = pd.merge(races, weather, how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country','weather'], axis = 1)
df2 = pd.merge(df1, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], axis = 1)

df3 = pd.merge(df2, driver_standings, how='left', on=['season', 'round', 'driver']) 
df4 = pd.merge(df3, constructor_standings, how='left', on=['season', 'round', 'constructor']) # from 1958

final_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_name', 'car'], axis = 1) # from 1983

In [27]:
print(final_df.shape)
final_df.head()

(14536, 22)


,season,round,circuit_id,date,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,...,constructor,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time
0,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,piquet,...,brabham,4,1,0.0,0.0,0.0,0.0,0.0,0.0,1:35.114
1,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,lauda,...,mclaren,9,2,0.0,0.0,0.0,0.0,0.0,0.0,1:36.054
2,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,laffite,...,williams,18,3,0.0,0.0,0.0,0.0,0.0,0.0,1:38.234
3,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,tambay,...,ferrari,3,4,0.0,0.0,0.0,0.0,0.0,0.0,1:34.758
4,1983,1,jacarepagua,1983-03-13,0,0,1,0,0,surer,...,arrows,20,5,0.0,0.0,0.0,0.0,0.0,0.0,1:38.468


## **4. Missing Values**

Before we can begin to deal with missing values, we need to identify which fields contain missing values:

In [29]:
final_df.isna().sum()

season                         0
round                          0
circuit_id                     0
date                           0
weather_warm                   0
weather_cold                   0
weather_dry                    0
weather_wet                    0
weather_cloudy                 0
driver                         0
date_of_birth                  0
nationality                    0
constructor                    0
grid                           0
podium                         0
driver_points                563
driver_wins                  563
driver_standings_pos         563
constructor_points           164
constructor_wins             164
constructor_standings_pos    164
qualifying_time              260
dtype: int64

As can be seen, it is only the final 7 fields that contain any missing data. For the fields with the most missing data (driver_points, driver_wins & driver_standing_pos), this is approximately only 3.9% of the data available. Therefore, any missing value imputation or removal should not have a significant effect on the final performance of the model 

In [31]:
# fill/drop nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    final_df[col].fillna(0, inplace = True)
    final_df[col] = final_df[col].map(lambda x: int(x))
    
final_df.dropna(inplace = True )